# Word2Vec: Obtain word embeddings

## 0. Introduction

**Word2vec** is the tool for generating the distributed representation of words, which is proposed by Mikolov et al[[1]](#1). When the tool assigns a real-valued vector to each word, the closer the meanings of the words, the greater similarity the vectors will indicate.

**Distributed representation** means assigning a real-valued vector for each object and representing the object by the vector. When representing a word by distributed representation, we call it **word embeddings**. In this tutorial, we will use the term.

Let's think about what the meaning of word is. Since we are human, so we can understand that the words "animal" and "dog" are related. But what information will Word2vec use in order to learn the vectors for meanings of words? The words "animal" and "dog" should have similar vectors, but the words "food" and "dog" should be far from each other.

## 1. Basic Idea

Word2vec learns the similarity of word meanings from simple information. It learns from a sequence of words in sentences. The idea is that the meaning of the word is determined by the words around it. This idea is based on **distributional hypothesis**[[2]](#2). The word to be learned is called the **"center word"**, and the words around it are called **"context words"**. Depending on the window size `c`, the number of Context Words will change.

Here, let's see the algorithm by using this example sentence: "**The cute cat jumps over the lazy dog.**"

- All of the following figures consider "cat" as the center word.
- According to the window size `c`, you can see that context words will change.

![](center_context_word.png)

## 2. Main Algorithm

Word2vec, the tool for creating the word embeddings, is actually built with two models, which are called **Skip-gram** and **CBoW**.

To explain the models with the figures below, we will use the following symbols.

| Symbol    | Definition                                               |
| --------: | :------------------------------------------------------- |
| $V$       | The size of vocabulary                                   |
| $D$       | The size of embedding vector                             |
| $v_t$     | A one-hot center word vector                             |
| $v_{t \pm c \backslash 0}$ | $c$ context vectors around $v_t$        |
| $l_H$     | An embedding vector of an input word vector              |
| $l_O$     | An output vector of the network                          |
| $W_H$     | The embedding matrix for inputs                          |
| $W_O$     | The embedding matrix for outputs                         |

**Note**

It is common to use **negative sampling** or **hierarchical softmax** for the loss function, however, in this tutorial, we will use the **softmax over all words** and skip the other variants because of simplifying the explanation.

### 2.1 Skip-gram

This model learns to predict context words $v_{t+c}$ when a center word $v_t$ is given. In the model, each row of the embedding matrix for input $W_H$ becomes a word embedding of each word.

When you input a center word $v_t$ into the network, you can calculate $\hat{v}_{t+c}$ as follows:

1. Calculate an embedding vector of the input center word vector: $l_H = W_H v_t$
2. Calculate an output vector of the embedding vector: $l_O = W_O l_H$
3. Calculate a probability vector of context words: $\hat{v}_{t+c} = \text{softmax}(l_O)$

Each element of the $V$-dimensional vector $\hat{v}_{t+c}$ is a probability that $i$-th word $w_i$ in the vocabulary is one of context words inside the $c$-sized window centered at the word $v_t$. So, the probability $p(v_{t+c} \mid v_t)$ can be calculated by a dot product of a one-hot word vector $v_{t+c}$ and the output vector $\hat{v}_{t+c}$.

$p(v_{t+c} \mid v_t) = v_{t+c}^T \hat{v}_{t+c}$

The loss function for the center word and the context words $\text{loss}(v_{t-c}, ..., v_t, ..., v_{t+c}; W_H, W_O)$ is,

$
\begin{eqnarray}
&&\text{loss}(v_{t-c}, ..., v_t, ..., v_{t+c}; W_H, W_O) \\
&=& \sum_{i=\{-c,...,c\}/\{0\}} -\log(p(v_{t+i}|v_t)) \\
&=& \sum_{i=\{-c,...,c\}/\{0\}} -\log(v_{t+i}^T \hat{v}_{t+i})
\end{eqnarray}
$

Let the training dataset be $\mathcal{D}=\{v_{t-c}^{(n)}, ..., v_t^{(n)}, ..., v_{t+c}^{(n)}\}_{n=1}^N$, the loss over the dataset $\text{Loss}(W_H, W_O|\mathcal{D})$ is,

$
\begin{eqnarray}
&&\text{Loss}(\mathcal{D}; W_H, W_O) \\
&=& \sum_{\mathcal{D}} \text{loss}(v_{t-c}, ..., v_t, ..., v_{t+c}; W_H, W_O) \\
&=& \sum_{\mathcal{D}} \sum_{i=\{-c,...,c\}/\{0\}} -\log(v_{t+i}^T \hat{v}_{t+c})
\end{eqnarray}
$

### 2.2 Continuous Bag of Words (CBoW)

This model learns to predict the center word $v_t$ when context words $v_{t+i}(i \in \{-c,\dots,c\}\backslash 0)$ is given. In this model, each column of the embedding matrix $W_O$ represents a word embedding vector of each word.

When you give a set of context words $v_{t+i}(i \in \{-c,\dots,c\}\backslash 0)$ to the network, you can calculate the probability of the center word $\hat{v}_t$ as follows:

1. Calculate a mean embedding vector from context words: $l_H = \frac{1}{2c} \sum_{i \in \{-c,...,c\}\backslash 0} W_H v_{t+i}$
2. Calculate an output vector: $l_O = W_O l_H$
3. Calculate an probability vector: $\hat{v}_t = \text{softmax}(l_O)$

Each element of $\hat{v}_t$ is a probability that the $i$-th word $w_i$ in the vocabulary is considered as the center word. So, the center word prediction $p(v_t \mid v_{t-c}, \dots v_{t-1}, v_{t+1}, \dots, v_{t+c})$ can be calculated by $v_t^T \hat{v}_t$, where $v_t$ denots the one-hot vector of the center word label vector.

The loss function for the center word prediction is,

$
\begin{eqnarray}
&& \text{loss}(v_t|v_{t \pm c \backslash 0}; W_H, W_O) \\
&=& \sum_{i \in \{-c,...,c\}\backslash 0} -\log(p(v_t|v_{t \pm C \backslash 0})) \\
&=& -\log(v_t^T \hat{v}_t)
\end{eqnarray}
$

Let the training dataset be $\mathcal{D}=\{v_{t-C}^{(n)}, \dots, v_t^{(n)}, \dots, v_{t+C}^{(n)}\}_{n=1}^N$, the loss functions for dataset $\text{Loss}(W_H, W_O|\mathcal{D})$ is,

$
\begin{eqnarray}
\text{Loss}(W_H, W_O|\mathcal{D})
&=& \sum_{\mathcal{D}} \text{loss}(W_H, W_O|v_{t-C}, ..., v_t, ..., v_{t+C}) \\
&=& \sum_{\mathcal{D}} \log(v_t^T v_t^*)
\end{eqnarray}
$

## 3. Details of Skip-gram

In this tutorial, we mainly explain Skip-gram from the following viewpoints.

1. It is easier to understand the algorithm than CBoW.
2. Even if the number of words increases, the accuracy is largely maintained. So, it is more scalable.

### 3.1 Example

In this example, we use the following setups.

* The size of vocabulary $N$ is 10.
* The size of embedding vector $D$ is 2.
* Center word is "dog".
* Context word is "animal".

Since there should be more than one Context Word, repeat the following process for each Context Word.

1. The one-hot vector of "dog" is `[0 0 1 0 0 0 0 0 0 0]` and you input it as Center Word.
2. After that, the third row of embedding matrix $W_H$ for Center Word is the word embedding of "dog" $L_H$.
3. The output layer $L_O$ is the result of multiplying the embedding matrix $W_O$ for Context Words by the embedding vector of "dog" $L_H$.
4. In order to limit the value of each element of the output layer,  softmax function is applied to the output layer $L_O$ to calculate $\text{softmax}(L_O)$. Softmax function normalizes scores in the output layer $L_O$ into sum 1 to see the scores as probability distribution over all words.
5. Calculate the error between $W_O$ and "animal"'s one-hot vector `[1 0 0 0 0 0 0 0 0 0 0]`, and propagate the error back to the network to update the parameters.

![](skipgram_detail.png)

## 4. Implementation of Skip-gram by Chainer

There is an example related to Word2vec on the GitHub repository, so we will explain based on that.

- [chainer/examples/word2vec](https://github.com/chainer/chainer/tree/master/examples/word2vec)

### 4.1 Implementation Method

First, let's import necessary packages:

In [1]:
import argparse
import collections

import numpy as np
import six

import chainer
from chainer import cuda
import chainer.functions as F
import chainer.initializers as I
import chainer.links as L
import chainer.optimizers as O
from chainer import reporter
from chainer import training
from chainer.training import extensions

* Basically, if you use chainer, you import in this way.
* Importing functions as ``F`` and links as ``L`` makes it easy to use.

#### Define Network Structures

Next, we define the network structures of skip-gram.

In [2]:
class SkipGram(chainer.Chain):

    def __init__(self, n_vocab, n_units, loss_func):
        super(SkipGram, self).__init__()

        with self.init_scope():
            self.embed = L.EmbedID(
                n_vocab, n_units, initialW=I.Uniform(1. / n_units))
            self.loss_func = loss_func

    def __call__(self, x, context):
        e = self.embed(context)
        shape = e.shape
        x = F.broadcast_to(x[:, None], (shape[0], shape[1]))
        e = F.reshape(e, (shape[0] * shape[1], shape[2]))
        x = F.reshape(x, (shape[0] * shape[1],))
        loss = self.loss_func(e, x)
        reporter.report({'loss': loss}, self)
        return loss

When we call the constructor `__init__`, we pass the vocabulary size `n_vocab`, the size of the embedding vector `n_units`, and the loss function `loss_func` as arguments.

The `Parameter`s are initialized in `self.init_scope()` It is recommended to initialize :class:`~chaier.Parameter` here. Since we set :class:`~chaier.Parameter` as the attribute of Link, there are effects such as making IDE easier to follow code.

For details, see [New-style parameter registration APIs are added to Link](https://docs.chainer.org/en/latest/upgrade.html#new-style-parameter-registration-apis-are-added-to-link).

The weight matrix `self.embed.W` is the embbeding matrix for input :math:`W_H`. The function call `__call__` takes Center Word's ID `x` and Context Word's ID `contexts` as arguments, and returns the error calculated by the loss function `self.loss_func`. When the function `__call__` is called, the shape of `x` is `[batch_size,]` and the shape of `contexts` is `[batch_size, n_context]`. The `batch_size` means the size of mini-batch, and `n_context` means the size of Context Words.

First, we obtain the embedding vectors of `contexts` by `e = self.embed(contexts)`. In the Skip-gram, since each Center Word has only one Context Word, there is no problem to switch Context Word and Center Word. So, in the code, Context Word is used as input for the network. (This is because it is easy to match the CBoW code.)

By `F.broadcast_to(x[:, None], (shape[0], shape[1]))`, the Center Word's ID `x` is broadcasted to each Context Word.

At the end, the shape of `x` is `[batch_size * n_context,]` and the shape of `e` is `[batch_size * n_context, n_units]`. By `self.loss_func(e, x)`, the error is calculated.

### Define Loss Function

Next, we define the loss function. Actually, this code also includes the part of the network structures.

In [3]:
class SoftmaxCrossEntropyLoss(chainer.Chain):

    def __init__(self, n_in, n_out):
        super(SoftmaxCrossEntropyLoss, self).__init__()
        with self.init_scope():
            self.out = L.Linear(n_in, n_out, initialW=0)

    def __call__(self, x, t):
        return F.softmax_cross_entropy(self.out(x), t)

After computing the linear transformation `self.out(x)`, which is defined by `L.Linear(n_in, n_out, initialW=0)`, we calculate the error function of cross entropy followed by softmax function with `softmax_cross_entropy`.

Here, the linear transformation matrix `self.out.W` corresponds to the embedding matrix for output $W_O$, and `softmax_cross_entropy` corresponds to the softmax function and the loss function.

### Define Iterator for Data

In [4]:
class WindowIterator(chainer.dataset.Iterator):

    def __init__(self, dataset, window, batch_size, repeat=True):
        self.dataset = np.array(dataset, np.int32)
        self.window = window
        self.batch_size = batch_size
        self._repeat = repeat

        self.order = np.random.permutation(
            len(dataset) - window * 2).astype(np.int32)
        self.order += window
        self.current_position = 0
        self.epoch = 0
        self.is_new_epoch = False

    def __next__(self):
        if not self._repeat and self.epoch > 0:
            raise StopIteration

        i = self.current_position
        i_end = i + self.batch_size
        position = self.order[i: i_end]
        w = np.random.randint(self.window - 1) + 1
        offset = np.concatenate([np.arange(-w, 0), np.arange(1, w + 1)])
        pos = position[:, None] + offset[None, :]
        context = self.dataset.take(pos)
        center = self.dataset.take(position)

        if i_end >= len(self.order):
            np.random.shuffle(self.order)
            self.epoch += 1
            self.is_new_epoch = True
            self.current_position = 0
        else:
            self.is_new_epoch = False
            self.current_position = i_end

        return center, context

    @property
    def epoch_detail(self):
        return self.epoch + float(self.current_position) / len(self.order)

    def serialize(self, serializer):
        self.current_position = serializer('current_position',
                                           self.current_position)
        self.epoch = serializer('epoch', self.epoch)
        self.is_new_epoch = serializer('is_new_epoch', self.is_new_epoch)
        if self._order is not None:
            serializer('_order', self._order)

- The constructor `__init__` receives the document dataset `dataset` as a list of word IDs, the window size `window` and the mini batch size `batch_size`.
    - In the constructor, we create an array `self.order` which is shuffled `[window, window + 1, ..., len(dataset) - window - 1]` in order to iterate randomly ordered `dataset`.
    - e.g. if the number of words in `dataset` is 100 and the window size `window` is 5, `self.order` becomes `numpy.ndarray` where numbers from 5 to 94 are shuffled.
- The iterator definition `__next__` returns mini batch sized Center Word `center` and Context Word `contexts` according to the parameters of the constructor.
    - The code `self.order[i:i_end]` generates the indices `position` of Center Words, which size is `batch_size`, from the random-ordered array `self.order`. The indices `position` will be converted to Center Words `center` by `self.dataset.take`.
    - The code `np.concatenate([np.arange (-w, 0), np.arange(1, w + 1)])` creates the window offset `offset`.
    - The code `position[:, None] + offset[None, :]` generates the indices of Context Words `pos` for each Center Word. The indices `pos` will be converted to Context Words `contexts` by `self.dataset.take`.

### Main Function

In [5]:
train, val, _ = chainer.datasets.get_ptb_words()
counts = collections.Counter(train)
counts.update(collections.Counter(val))
n_vocab = max(train) + 1

- `train` and `val` means training data and validation data. Each data contains the list of Document IDs

```
>>> train
array([ 0,  1,  2, ..., 39, 26, 24], dtype=int32)
>>> val
array([2211,  396, 1129, ...,  108,   27,   24], dtype=int32)
```

- The maximum id in `train` will be the vocabulary size `n_vocab - 1`.

In [9]:
unit = 100
loss_func = SoftmaxCrossEntropyLoss(unit, n_vocab)
model = SkipGram(n_vocab, unit, loss_func)

We create the `model` as Skip-gram.

We create the loss function `loss_func` as `SoftmaxCrossEntropyLoss`.

In [11]:
optimizer = O.Adam()
optimizer.setup(model)

We create the `optimizer` as Adam (Adaptive moment estimation).

In [16]:
window = 5
batchsize = 1000
gpu = -1
train_iter = WindowIterator(train, window, batchsize)
val_iter = WindowIterator(val, window, batchsize, repeat=False)

updater = training.StandardUpdater(
    train_iter, optimizer, device=gpu)

We create the iterators and updater.

In [20]:
epoch = 10
trainer = training.Trainer(
    updater, (epoch, 'epoch'), out='word2vec_result')
trainer.extend(extensions.Evaluator(
    val_iter, model, device=gpu))
trainer.extend(extensions.LogReport())
trainer.extend(extensions.PrintReport(
    ['epoch', 'main/loss', 'validation/main/loss']))
trainer.extend(extensions.ProgressBar())

We create the trainer and run it.

In [21]:
trainer.run()

Exception in main training loop: all the input arrays must have same number of dimensions
Traceback (most recent call last):
  File "/Users/shunta/lib/chainer/chainer/training/trainer.py", line 310, in run
    update()
  File "/Users/shunta/lib/chainer/chainer/training/updater.py", line 223, in update
    self.update_core()
  File "/Users/shunta/lib/chainer/chainer/training/updater.py", line 228, in update_core
    in_arrays = self.converter(batch, self.device)
  File "/Users/shunta/lib/chainer/chainer/dataset/convert.py", line 109, in concat_examples
    return to_device(device, _concat_arrays(batch, padding))
  File "/Users/shunta/lib/chainer/chainer/dataset/convert.py", line 123, in _concat_arrays
    return xp.concatenate([array[None] for array in arrays])
Will finalize trainer extensions and updater before reraising the exception.


ValueError: all the input arrays must have same number of dimensions